In [1]:
import os
import io

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import numpy as np
import random
from PIL import Image
from keras.applications.mobilenet import MobileNet, preprocess_input
from keras.callbacks import Callback
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.utils.generic_utils import CustomObjectScope
import tensorflow as tf
import keras.backend.tensorflow_backend as tfb
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
from keras.layers import DepthwiseConv2D, ReLU
relu6 = ReLU(6.)
import copy
from keras.models import load_model

Using TensorFlow backend.


In [2]:
def load_numpy(datapath, encoding='bytes'):
    return np.load(datapath, encoding=encoding)

def prepare_items(items):
    item_list = {}
    for i in range(itemnum):
        item = items[i]
        category = get_categories_from_item(item)
        img = item[b'imgs']
        item_list[item[b'asin'].decode('utf8')] = (img, category)
    return item_list

def build_category_dict(items, min_number_of_items_per_category):
    categories = set()
    for i in range(itemnum):
        for cat_list in items[i][b'categories']:
            for cat in cat_list:
                if cat.decode('utf8').lower() not in stop_categories and len(cat.decode('utf8')) > 1: categories.add(cat.decode('utf8').lower())
    categories = dict.fromkeys(list(categories), 0)
    for i in range(itemnum):
        for cat_list in items[i][b'categories']:
            for cat in cat_list:
                if cat.decode('utf8').lower() in categories: categories[cat.decode('utf8').lower()] += 1
    sorted_cats = sorted(categories.items(), key=lambda kv: kv[1], reverse=True)
    top_cats = [c[0] for c in sorted_cats if c[1] >= min_number_of_items_per_category]
    return top_cats, {top_cats[i]: i for i in range(len(top_cats))}

def get_categories_from_item(item):
    item_categories = set()
    for cat_list in item[b'categories']:
        for cat in cat_list:
            if cat.decode('utf8').lower() in categories: item_categories.add(cat.decode('utf8').lower())
    return list(item_categories)

def group_items_by_user(users):
    userlist = []
    max_items = 0
    items_sum = 0
    for i in range(len(users)):
        u = users[i]
        user_items = []
        for j in range(len(u)):
            user_items.append(u[j]['asin'])
        if len(user_items) > max_items:
            max_items = len(user_items)
        items_sum += len(u)
        userlist.append(copy.deepcopy(user_items))
    max_items = max_items
    average_items = int(round(items_sum / len(users)) * 2)
    userlist = list(filter(lambda x: len(x) <= average_items, userlist))
    #print("length of userlist is %d" % len(userlist))
    return userlist

def load_image(img_bytes):
    return Image.open(io.BytesIO(img_bytes)).convert('RGB').resize((224, 224))

def save_embeddings():
    with open(preextracted_path, 'wb') as f:
        pickle.dump(extracted_imgs, f)

def load_embeddings():
    with open(preextracted_path, 'rb') as f:
        ret_di = pickle.load(f)
    return ret_di

def cat2idx(categories):
    cat_indices = [category_dict[c] for c in categories]
    return np.array([1 if i in cat_indices else 0 for i in range(num_categories)])

def get_image_embedding(img):
    x = preprocess_input(np.array(img))
    x = np.expand_dims(x, axis=0)
    features = features_model.predict(x)
    return np.squeeze(features)

def extract_features(idx, users):
    user = users[idx]
    imgs = []
    categories = []
    for i in range(min(len(user), 8)):
        asin = user[i]
        img, cat = items[asin]
        if asin in extracted_imgs:
            img = extracted_imgs[asin]
        else:
            img = get_image_embedding(load_image(img))
            #extracted_imgs[asin] = img.numpy() change to tf
        cat = cat2idx(cat)
        imgs.append(img)
        categories.append(cat)

    for i in range(8 - len(imgs)):
        imgs.append(np.zeros_like(imgs[0]))
    category_tensor = np.clip(np.sum(np.array(categories), axis=0), 0, 1)

    return np.array(imgs), category_tensor

def create_model():
    base_model = MobileNet(
        weights='imagenet',
        include_top=False,
        input_shape=(224, 224, 3)
    )
    top_layer = base_model.output
    top_layer = GlobalAveragePooling2D()(top_layer)
    top_layer = Dropout(rate=0.5)(top_layer)
    predictions = Dense(NUM_CLASSES, activation='sigmoid')(top_layer)
    return Model(inputs=base_model.input, outputs=predictions)

In [3]:
datapath = "/home/kdemochkin/2019_research/UserVisualPreferences/data/AmazonFashion6ImgPartitioned.npy"
stop_categories = ['shoes & accessories: international shipping available',
           'clothing, shoes & jewelry',
           'boys',
           'men',
           'women',
           'girls',
           'clothing',
           'novelty, costumes & more',
           'jewelry: international shipping available',
           'available for snternational shipping']

[user_train, user_validation, user_test, Item, usernum, itemnum] = load_numpy(datapath)
categories, category_dict = build_category_dict(Item, 1000)
num_categories = len(categories)
items = prepare_items(Item)
users_train = group_items_by_user(user_train)
users_test = group_items_by_user(user_test)

In [4]:
NUM_CLASSES = num_categories
model = create_model()
model.load_weights('res/final_weights.hdf5')
layer_name = 'global_average_pooling2d_1'
features_model = Model(
    inputs=model.input,
    outputs=model.get_layer(layer_name).output)
print(features_model.summary())
for layer in features_model.layers:
    layer.trainable = False

extracted_imgs = {}


Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128 

In [5]:
np_x_train = []
np_x_test = []
np_y_train = []
np_y_test = []

for i in range(len(users_train)):
    x, y = extract_features(i, users_train)
    np_x_train.append(x)
    np_y_train.append(y)
    if i % 1000 == 0:
        print(i)

np.save('ag_train_x', np.array(np_x_train))
np.save('ag_train_y', np.array(np_y_train))
        
for i in range(len(users_test)):
    x, y = extract_features(i, users_test)
    np_x_test.append(x)
    np_y_test.append(y)
    if i % 1000 == 0:
        print(i)

np.save('test_agg_x',  np.array(np_x_test))
np.save('test_agg_y',  np.array(np_y_test))

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000


In [6]:
with open('labels.txt', 'w+') as f:
    for item in categories:
        f.write("%s\n" % item)